In [1]:
import itertools
import numpy as np
from dataclasses import dataclass
from typing import Tuple, List, Optional

from scipy.optimize import linprog, minimize
from scipy.sparse import coo_matrix, csr_matrix


# --------------------------
# Utilities
# --------------------------

def all_bitstrings(n: int) -> np.ndarray:
    return np.array(list(itertools.product([0, 1], repeat=n)), dtype=int)


def all_branches(n: int, h: int) -> List[Tuple[Tuple[int, ...], Tuple[int, ...]]]:
    k = n - h
    branches = []
    for A in itertools.combinations(range(n), k):
        for aA in itertools.product([0, 1], repeat=k):
            branches.append((tuple(A), tuple(aA)))
    return branches


def sigmoid(z):
    z = np.clip(z, -40, 40)
    return 1.0 / (1.0 + np.exp(-z))


def sort_desc(s: np.ndarray) -> np.ndarray:
    return np.sort(np.asarray(s, dtype=float))[::-1]


def linprog_any(c, A_ub, b_ub, A_eq, b_eq, bounds):
    methods = ("highs", "highs-ds", "highs-ipm", "interior-point", "revised simplex", "simplex")
    last_err = None
    for method in methods:
        try:
            return linprog(
                c=c, A_ub=A_ub, b_ub=b_ub,
                A_eq=A_eq, b_eq=b_eq,
                bounds=bounds,
                method=method
            )
        except ValueError as e:
            last_err = e
            continue
    if last_err is not None:
        raise last_err
    raise RuntimeError("linprog failed unexpectedly.")


def cache_key_from_s(s: np.ndarray, *, tol: float = 1e-4) -> tuple:
    s = np.clip(np.asarray(s, dtype=float), 0.0, 1.0)
    q = np.round(s / tol).astype(np.int64)
    diffs = np.diff(q)
    return (int(q[0]), *map(int, diffs))


# --------------------------
# Inner LP solver
# --------------------------

@dataclass
class LPValue:
    ok: bool
    t: float
    f: Optional[np.ndarray]
    msg: str


class FullProgramLPSolver:
    """
    Fixes the build_ub bottleneck:
      - precompute A_ub CSR structure ONCE
      - per solve, only fill COO-order data vector then permute into CSR-order data
      - reuse indices/indptr (no COO->CSR conversion per miss)

    Also uses cheap weight computation from term = where(bits, s, 1-s):
      - pi = prod(term, axis=1)
      - eq weights: w_i = pi / term[:, i]
      - branch weights: w_A = prod(term[:, comp(A)], axis=1)   where comp(A) has size h
    """

    def __init__(
        self,
        n: int,
        h: int,
        C: float,
        cache_tol: float = 1e-4,
        cache_max: int = 200_000,
        profile: bool = True,
    ):
        self.n = int(n)
        self.h = int(h)
        self.C = float(C)

        self.bits = all_bitstrings(self.n)  # (m,n)
        self.m = self.bits.shape[0]
        self.all_zero = (self.bits.sum(axis=1) == 0).astype(float)  # (m,)

        self.branches = all_branches(self.n, self.h)
        self.num_branches = len(self.branches)

        # Variable layout
        self.N = self.n * self.m + 1
        self.t_idx = self.n * self.m
        self.bounds = [(0.0, None)] * (self.n * self.m) + [(0.0, None)]
        self.c = np.zeros(self.N, dtype=float)
        self.c[self.t_idx] = 1.0

        # Precompute eq index sets
        self.eq_idx1 = [np.where(self.bits[:, i] == 1)[0] for i in range(self.n)]
        self.eq_idx0 = [np.where(self.bits[:, i] == 0)[0] for i in range(self.n)]
        self.eq_cols1 = [i * self.m + self.eq_idx1[i] for i in range(self.n)]
        self.eq_cols0 = [i * self.m + self.eq_idx0[i] for i in range(self.n)]

        # Branch precompute:
        # - idx of a matching (A,aA)
        # - comp(A) columns (size h)
        # - whether idx contains 0 (all-zero bitstring, which is row 0)
        all_idx = np.arange(self.n)
        self.branch_match_idx: List[np.ndarray] = []
        self.branch_cols_per_i: List[List[np.ndarray]] = []
        self.branch_comp_cols: List[np.ndarray] = []
        self.branch_has_zero = np.zeros(self.num_branches, dtype=bool)

        for b, (A, aA) in enumerate(self.branches):
            mask = np.ones(self.m, dtype=bool)
            for pos, j in enumerate(A):
                mask &= (self.bits[:, j] == aA[pos])
            idx = np.where(mask)[0]
            self.branch_match_idx.append(idx)
            self.branch_cols_per_i.append([i * self.m + idx for i in range(self.n)])

            Aset = set(A)
            comp = np.array([j for j in all_idx if j not in Aset], dtype=int)  # size h
            self.branch_comp_cols.append(comp)

            self.branch_has_zero[b] = (idx.size > 0 and idx[0] == 0) or (0 in set(idx.tolist()))

        # ----------------------
        # Precompute A_ub pattern in COO order (rows, cols) + slices
        # ----------------------
        self.num_ub = 1 + self.num_branches

        total_nnz = self.n * self.m
        for idx in self.branch_match_idx:
            total_nnz += self.n * idx.size + 1

        self._ub_rows = np.empty(total_nnz, dtype=int)
        self._ub_cols = np.empty(total_nnz, dtype=int)

        self._ir_slices: List[slice] = []
        self._branch_slices: List[List[slice]] = []
        self._t_pos = np.empty(self.num_branches, dtype=int)

        off = 0
        # IR row pattern (row 0)
        for i in range(self.n):
            cols = i * self.m + np.arange(self.m, dtype=int)
            self._ub_rows[off:off + self.m] = 0
            self._ub_cols[off:off + self.m] = cols
            self._ir_slices.append(slice(off, off + self.m))
            off += self.m

        # Branch row patterns
        for b in range(self.num_branches):
            row = 1 + b
            idx = self.branch_match_idx[b]
            b_slices = []
            for i in range(self.n):
                cols = self.branch_cols_per_i[b][i]
                L = cols.size
                self._ub_rows[off:off + L] = row
                self._ub_cols[off:off + L] = cols
                b_slices.append(slice(off, off + L))
                off += L

            # -t entry
            self._ub_rows[off] = row
            self._ub_cols[off] = self.t_idx
            self._t_pos[b] = off
            off += 1
            self._branch_slices.append(b_slices)

        assert off == total_nnz

        # ----------------------
        # Build CSR template ONCE + permutation mapping COO-order -> CSR-order
        # Trick: put data = 0..nnz-1, convert to CSR, then csr.data are those ids in CSR order.
        # ----------------------
        ids = np.arange(total_nnz, dtype=float)
        csr_template = coo_matrix((ids, (self._ub_rows, self._ub_cols)),
                                  shape=(self.num_ub, self.N)).tocsr()

        # perm[k] = which COO entry sits at CSR position k
        self._ub_perm = csr_template.data.astype(np.int64)
        self._ub_indices = csr_template.indices.copy()
        self._ub_indptr = csr_template.indptr.copy()

        # A_eq is tiny; keep building it per miss (it wasn't your bottleneck)

        # Cache
        self.cache_tol = float(cache_tol)
        self._cache: dict[tuple, LPValue] = {}
        self._cache_max = int(cache_max)

        # Profiling
        self.profile = bool(profile)
        self._prof = {
            "calls": 0,
            "cache_hits": 0,
            "build_eq_s": 0.0,
            "build_ub_s": 0.0,
            "linprog_s": 0.0,
            "total_s": 0.0,
        }

    def solve_lp(self, s: np.ndarray, return_f: bool = False) -> LPValue:
        import time
        t0 = time.perf_counter()
        if self.profile:
            self._prof["calls"] += 1

        s = np.clip(np.asarray(s, dtype=float), 0.0, 1.0)

        key = cache_key_from_s(s, tol=self.cache_tol)
        cached = self._cache.get(key)
        if cached is not None and ((not return_f) or (cached.f is not None)):
            if self.profile:
                self._prof["cache_hits"] += 1
                self._prof["total_s"] += time.perf_counter() - t0
            return cached

        n, m, C = self.n, self.m, self.C
        N = self.N

        # Precompute term + pi once
        s_safe = np.clip(s, 1e-12, 1 - 1e-12)
        term = np.where(self.bits == 1, s_safe[None, :], (1.0 - s_safe)[None, :])  # (m,n)
        pi = term.prod(axis=1)  # (m,)

        # ----------------------
        # Build A_eq (sparse)
        # ----------------------
        t_eq0 = time.perf_counter()
        eq_rows = []
        eq_cols = []
        eq_data = []

        for i in range(n):
            w_i = pi / term[:, i]
            idx1 = self.eq_idx1[i]
            idx0 = self.eq_idx0[i]
            cols1 = self.eq_cols1[i]
            cols0 = self.eq_cols0[i]

            eq_rows.append(np.full(cols1.shape[0], i, dtype=int))
            eq_cols.append(cols1.astype(int))
            eq_data.append(w_i[idx1])

            eq_rows.append(np.full(cols0.shape[0], i, dtype=int))
            eq_cols.append(cols0.astype(int))
            eq_data.append(-w_i[idx0])

        A_eq = coo_matrix(
            (np.concatenate(eq_data), (np.concatenate(eq_rows), np.concatenate(eq_cols))),
            shape=(n, N),
        ).tocsr()
        b_eq = np.ones(n, dtype=float)

        if self.profile:
            self._prof["build_eq_s"] += time.perf_counter() - t_eq0

        # ----------------------
        # Build A_ub by ONLY filling data (no COO->CSR conversion)
        # ----------------------
        t_ub0 = time.perf_counter()

        data_coo = np.empty_like(self._ub_rows, dtype=float)
        b_ub = np.zeros(self.num_ub, dtype=float)

        # IR row blocks: fill -pi
        for sl in self._ir_slices:
            data_coo[sl] = -pi
        b_ub[0] = -float(s.sum())

        # Branch rows
        for b in range(self.num_branches):
            comp = self.branch_comp_cols[b]  # size h
            if comp.size == 0:
                w = np.ones(m, dtype=float)
            else:
                w = term[:, comp].prod(axis=1)  # (m,)

            idx = self.branch_match_idx[b]
            w_idx = w[idx]

            for i in range(n):
                data_coo[self._branch_slices[b][i]] = w_idx

            data_coo[self._t_pos[b]] = -1.0

            # RHS constant uses only the all-zero assignment (bitstring 000...0 is index 0)
            if self.branch_has_zero[b]:
                b_ub[1 + b] = -C * float(w[0])
            else:
                b_ub[1 + b] = 0.0

        # Reorder data into CSR order using perm computed once
        data_csr = data_coo[self._ub_perm]

        A_ub = csr_matrix((data_csr, self._ub_indices, self._ub_indptr), shape=(self.num_ub, self.N))

        if self.profile:
            self._prof["build_ub_s"] += time.perf_counter() - t_ub0

        # ----------------------
        # Solve LP
        # ----------------------
        t_lp0 = time.perf_counter()
        res = linprog_any(self.c, A_ub, b_ub, A_eq, b_eq, self.bounds)
        if self.profile:
            self._prof["linprog_s"] += time.perf_counter() - t_lp0

        if not res.success:
            val = LPValue(ok=False, t=float("inf"), f=None, msg=str(res.message))
            if len(self._cache) < self._cache_max:
                self._cache[key] = val
            if self.profile:
                self._prof["total_s"] += time.perf_counter() - t0
            return val

        t_val = float(res.fun)
        if not return_f:
            val = LPValue(ok=True, t=t_val, f=None, msg="OK")
            if len(self._cache) < self._cache_max:
                self._cache[key] = val
            if self.profile:
                self._prof["total_s"] += time.perf_counter() - t0
            return val

        x = res.x
        f = x[: n * m].reshape((n, m))
        val = LPValue(ok=True, t=t_val, f=f, msg="OK")
        if len(self._cache) < self._cache_max:
            self._cache[key] = val
        if self.profile:
            self._prof["total_s"] += time.perf_counter() - t0
        return val

    def print_profile(self, label: str = ""):
        p = self._prof
        calls = p["calls"]
        hits = p["cache_hits"]
        misses = calls - hits
        head = f"=== solve_lp profile {label} ===".strip()
        print("\n" + head)
        print(f"calls:      {calls}")
        print(f"cache_hits: {hits} ({hits / max(1, calls):.1%})")
        print(f"misses:     {misses} ({misses / max(1, calls):.1%})")
        print(f"build_eq:   {p['build_eq_s']:.3f}s")
        print(f"build_ub:   {p['build_ub_s']:.3f}s")
        print(f"linprog:    {p['linprog_s']:.3f}s")
        print(f"total:      {p['total_s']:.3f}s")



In [2]:
# --------------------------
# Outer optimization (NO faces) with blocky + random starts
# --------------------------

from dataclasses import dataclass
import numpy as np
from scipy.optimize import minimize
from typing import List

@dataclass
class SearchResult:
    ok: bool
    t: float
    s: np.ndarray
    msg: str


def s_from_z_monotone(z: np.ndarray) -> np.ndarray:
    """
    Smooth monotone map R^n -> {1 >= s1 >= ... >= sn >= 0} without sorting.
    s1 = sigmoid(z1)
    si = s_{i-1} * sigmoid(zi)
    """
    z = np.asarray(z, dtype=float)
    s = np.empty_like(z)
    s[0] = sigmoid(z[0])
    for i in range(1, z.size):
        s[i] = s[i - 1] * sigmoid(z[i])
    return s


def z_from_s_monotone(s: np.ndarray) -> np.ndarray:
    """
    Inverse-ish map for initialization:
      z1 = logit(s1)
      zi = logit(si / s_{i-1}) for i>=2
    """
    s = np.clip(np.asarray(s, dtype=float), 1e-8, 1 - 1e-8)

    def logit(u):
        u = np.clip(u, 1e-8, 1 - 1e-8)
        return np.log(u / (1 - u))

    z = np.empty_like(s)
    z[0] = logit(s[0])
    for i in range(1, s.size):
        ratio = s[i] / max(s[i - 1], 1e-12)
        z[i] = logit(np.clip(ratio, 1e-8, 1 - 1e-8))
    return z


def blocky_seeds(n: int, rng: np.random.Generator) -> List[np.ndarray]:
    """
    Generate monotone, 'face-like' s vectors as STARTS (not constraints).
    Includes:
      - spike + flat tail: [1, u, ..., u]
      - two-block / three-block step functions
      - stick-breaking sharp drops
    """
    seeds: list[np.ndarray] = []

    # 1) Spike + flat tail
    for u in np.linspace(0.02, 0.8, 25):
        s = np.full(n, float(u))
        s[0] = 1.0
        seeds.append(s)

    # 2) Slightly relaxed spike (helps smooth local methods)
    for u in np.linspace(0.02, 0.8, 12):
        for top in (0.98, 0.995, 0.999):
            s = np.full(n, float(u))
            s[0] = float(top)
            seeds.append(s)

    # 3) Two-block steps
    for k in range(1, n):
        for _ in range(10):
            a = rng.uniform(0.6, 1.0)
            b = rng.uniform(0.0, min(a, 0.7))
            s = np.empty(n)
            s[:k] = a
            s[k:] = b
            seeds.append(s)

    # 4) Three-block steps
    for _ in range(40):
        k1 = int(rng.integers(1, n - 1))
        k2 = int(rng.integers(1, n - k1))
        a = rng.uniform(0.7, 1.0)
        b = rng.uniform(0.15, min(a, 0.85))
        c = rng.uniform(0.0, min(b, 0.6))
        s = np.empty(n)
        s[:k1] = a
        s[k1:k1 + k2] = b
        s[k1 + k2:] = c
        seeds.append(s)

    # 5) Stick-breaking monotone (often produces sharp drops)
    for _ in range(60):
        s = np.empty(n)
        s[0] = rng.uniform(0.5, 1.0)
        for i in range(1, n):
            s[i] = s[i - 1] * rng.uniform(0.0, 1.0)
        seeds.append(s)

    return seeds


def minimize_over_s_unconstrained(
    solver: FullProgramLPSolver,
    n_random: int = 300,
    n_blocky_starts: int = 80,
    n_random_starts: int = 12,
    seed: int = 0,
    local_method: str = "Powell",
    maxiter: int = 600,
    add_best_scan_start: bool = True,
) -> SearchResult:
    n = solver.n
    rng = np.random.default_rng(seed)

    # --- phase 1: random scan in z-space -> monotone s ---
    best_t = float("inf")
    best_s = None

    for _ in range(n_random):
        z = rng.normal(size=n)
        s = s_from_z_monotone(z)
        val = solver.solve_lp(s, return_f=False)
        if val.ok and val.t < best_t:
            best_t = val.t
            best_s = s.copy()

    if best_s is None:
        best_s = np.full(n, 0.5, dtype=float)

    # --- objective in z-space ---
    def obj(z):
        s = s_from_z_monotone(z)
        val = solver.solve_lp(s, return_f=False)
        return val.t if val.ok else 1e6

    # --- build starts: (1) best from scan, (2) blocky, (3) random ---
    starts: list[np.ndarray] = []

    if add_best_scan_start:
        starts.append(z_from_s_monotone(best_s))

    # blocky starts (shuffle & cap)
    bseeds = blocky_seeds(n, rng)
    rng.shuffle(bseeds)
    for s0 in bseeds[: max(0, n_blocky_starts)]:
        starts.append(z_from_s_monotone(s0))

    # random starts
    for _ in range(max(0, n_random_starts)):
        starts.append(rng.normal(size=n))

    # --- local searches ---
    best_z = None
    best_local = best_t
    best_msg = "OK"

    for z0 in starts:
        res = minimize(obj, z0, method=local_method, options={"maxiter": maxiter, "disp": False})
        f = float(res.fun)
        if f < best_local:
            best_local = f
            best_z = np.array(res.x, copy=True)

    s_star = s_from_z_monotone(best_z) if best_z is not None else best_s
    lp_star = solver.solve_lp(s_star, return_f=False)
    return SearchResult(lp_star.ok, lp_star.t, s_star, lp_star.msg)

In [ ]:
n = 8
C = 4.0

outer_kwargs = dict(
    n_random=250,          # scan budget
    n_blocky_starts=120,   # blocky starts help find face-like optima
    n_random_starts=10,    # still include some random starts
    seed=1,
    local_method="Powell",
    maxiter=500,
)

for h in range(1, n):
    print(f"\n{'='*60}")
    print(f"Solving n={n}, h={h}, C={C}")

    solver = FullProgramLPSolver(
        n=n,
        h=h,
        C=C,
        cache_tol=1e-4,
        cache_max=200_000,
        profile=True,
    )

    res = minimize_over_s_unconstrained(solver, **outer_kwargs)

    print(
        f"\nRESULT (NO faces) n={n}, h={h}, C={C}\n"
        f"  t={res.t: .6g}\n"
        f"  s={np.array2string(res.s, precision=6, floatmode='fixed')}\n"
        f"  ok={res.ok}, msg={res.msg}"
    )

    solver.print_profile(label=f"(n={n}, h={h}, C={C})")



Solving n=8, h=1, C=4.0

RESULT (NO faces) n=8, h=1, C=4.0
  t= 2.90938
  s=[1.000000 0.272746 0.272746 0.272746 0.272746 0.272746 0.272746 0.272746]
  ok=True, msg=OK

=== solve_lp profile (n=8, h=1, C=4.0) ===
calls:      50378
cache_hits: 37137 (73.7%)
misses:     13241 (26.3%)
build_eq:   5.140s
build_ub:   113.466s
linprog:    316.363s
total:      437.783s

Solving n=8, h=2, C=4.0

RESULT (NO faces) n=8, h=2, C=4.0
  t= 2.48395
  s=[1.000000 0.211963 0.211963 0.211963 0.211963 0.211963 0.211963 0.211963]
  ok=True, msg=OK

=== solve_lp profile (n=8, h=2, C=4.0) ===
calls:      58752
cache_hits: 43972 (74.8%)
misses:     14780 (25.2%)
build_eq:   5.779s
build_ub:   236.609s
linprog:    1367.260s
total:      1612.906s

Solving n=8, h=3, C=4.0


In [9]:
def blocky_seeds(n: int, rng: np.random.Generator):
    """
    Yield monotone 'face-like' s vectors as GOOD STARTS (not constraints).
    Includes:
      - [1, u, u, ..., u]  (your best h=4,n=6 case)
      - 2-3 block step functions
      - near-1 spike with flat tail
    """
    seeds = []

    # 1) Spike + flat tail: s = [1, u, ..., u]
    for u in np.linspace(0.05, 0.8, 20):
        s = np.full(n, u, dtype=float)
        s[0] = 1.0
        seeds.append(s)

    # 2) Slightly relaxed spike (helps smooth optimizers)
    for u in np.linspace(0.05, 0.8, 10):
        for top in (0.98, 0.995, 0.999):
            s = np.full(n, u, dtype=float)
            s[0] = top
            seeds.append(s)

    # 3) Two-block step: [a,...,a, b,...,b] with a>=b
    for k in range(1, n):  # split point
        for _ in range(8):
            a = rng.uniform(0.6, 1.0)
            b = rng.uniform(0.0, min(a, 0.6))
            s = np.empty(n)
            s[:k] = a
            s[k:] = b
            seeds.append(s)

    # 4) Three-block step: [a]*k1 + [b]*k2 + [c]*rest
    for _ in range(30):
        k1 = rng.integers(1, n-1)
        k2 = rng.integers(1, n-k1)
        a = rng.uniform(0.7, 1.0)
        b = rng.uniform(0.2, min(a, 0.8))
        c = rng.uniform(0.0, min(b, 0.5))
        s = np.empty(n)
        s[:k1] = a
        s[k1:k1+k2] = b
        s[k1+k2:] = c
        seeds.append(s)

    # 5) Random "stick-breaking" monotone (often produces sharp drops)
    for _ in range(50):
        s = np.empty(n)
        s[0] = rng.uniform(0.5, 1.0)
        for i in range(1, n):
            s[i] = s[i-1] * rng.uniform(0.0, 1.0)
        seeds.append(s)

    return seeds
